## Select 심화

오늘은 기본적으로 `select` 를 이용하는 내용이 아닌 조금 더 딥하게 다뤄보겠습니다. 

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session

class Base(DeclarativeBase):
    """
    DeclarativeBase 를 상속 받은 Base 라는 하위 클래스를 만들고 시작.
    이 Base 에 Mapping 된 클래스들은 database 에서 단일 테이블임.

    `__tablename__` 을 클래스 레벨의 속성으로 지녀야 함.
    """
    pass

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

Base.metadata.create_all(engine)  
    

In [ ]:
from sqlalchemy.orm import Session

def generate_100_users():
    with Session(engine) as session:
        for i in range(100):
            session.add(User(name=f"user{i}", fullname=f"User {i}", addresses=[Address(email_address=f"user{i}@example.com")]))
        session.flush()
        session.commit()

generate_100_users()  

## Result 와 Row

SQLAlchemy 의 공식문서를 읽어보면 Session.execute() 는 2.0 버전에서 ORM 에서 statement 를 실행시키기 위해 알아두어야 할 중요할점이라고 적어두었는데요. 문서를 읽어보면 `Session.execute()` 에서는 `Result` 오브젝트를 반환한다고 적혀있습니다. 중요한 만큼 `Result` 에 대해서도 알아봐야 할거 같은데요. 과연 `Result` 객체는 어떤 역할을 할까요?

In [ ]:
from sqlalchemy import select

with Session(engine) as session:
    stmt = select(User)
    users = session.execute(stmt)
    print(users) # sqlalchemy.engine.result.ChunkedIteratorResult
    print(type(users)) # class 'sqlalchemy.engine.result.ChunkedIteratorResult'

실제로 위 코드를 실행해보니 `ChunkedIteratorResult` 타입의 객체를 반환합니다. 이 객체는 무엇일까요? 공식문서에서는 아래와 같이 설명하고 있습니다.
사실 `Result` 자체는 쿼리 수행으로 얻어지는 추상적인 표현입니다. 정확히 보기 위해 실제 코드를 잠시 가져와 보겠습니다.

```python
class Result(_WithKeys, ResultInternal[Row[Unpack[_Ts]]]):
    """Represent a set of database results.
    When using the ORM, a higher level object called :class:`_engine.ChunkedIteratorResult` is normally used.
    """
    @property
    def _soft_closed(self) -> bool:
        raise NotImplementedError()

    @property
    def closed(self) -> bool:
        raise NotImplementedError()
```

확인해보시면 주요 메소드들이 `NotImplementedError` 예외를 던지는 것을 확인할 수 있습니다. 즉, **Interface** 역할을 하고 있음을 알 수 있습니다.
class 에 달려있는 메소드를 찾아보면 즉, 우리가 받았던 `ChunkedIteratorResult` 객체를 반환한다는 것을 확인할 수 있습니다.
이제 왜 `ChunkedIteratorResult` 객체가 나왔는지 알았으니 이걸로 무엇을 할 수 있는지 찾아보도록 합시다.

일단은 기본적으로 Result 를 상속하고 있기에 아래에 보이는 Result 의 메소드들은 대부분 사용이 가능합니다. 일단 이 강의가 심화과정인 만큼 데이터를 가져오는 메소드 들을 포함해서 모든 메소드들을 실제 코드와 함께 둘러보도록 하겠습니다.

```
all(), close(), closed, columns(), fetchall(), fetchmany(), fetchone(), first(), freeze(), keys(), mappings(), merge(), one(), one_or_none(), partitions(), scalar(), scalar_one(), scalar_one_or_none(), scalars(), t, tuples(), unique(), yield_per()
```

일단 `all()` 같은 경우

```python
    def all(self) -> Sequence[Row[_TP]]:
        """Return all rows in a sequence.

        Closes the result set after invocation.   Subsequent invocations
        will return an empty sequence.

        .. versionadded:: 1.4

        :return: a sequence of :class:`_engine.Row` objects.

        .. seealso::

            :ref:`engine_stream_results` - How to stream a large result set
            without loading it completely in python.

        """

        return self._allrows()

    def _allrows(self) -> List[_R]:
        # 후처리 필터
        post_creational_filter = self._post_creational_filter

        # _row_getter => @HasMemoized_ro_memoized_attribute 를 통해 접근할때 한번만 계산하고 그 이후에는 메모리에 캐시
        make_row = self._row_getter

        # 모든 데이터 가져오기
        rows = self._fetchall_impl()
        made_rows: List[_InterimRowType[_R]]

        # 가져온 데이터를 Row 객체 또는 지정된 타입으로 변환
        if make_row:
            made_rows = [make_row(row) for row in rows]
        else:
            made_rows = rows  # type: ignore

        interim_rows: List[_R]

        # unique 필터가 있다면 지정된 타입으로 변환
        if self._unique_filter_state:
            uniques, strategy = self._unique_strategy

            interim_rows = [
                made_row  # type: ignore
                for made_row, sig_row in [
                    (
                        made_row,
                        strategy(made_row) if strategy else made_row,
                    )
                    for made_row in made_rows
                ]
                if sig_row not in uniques and not uniques.add(sig_row)  # type: ignore # noqa: E501
            ]
        else:
            interim_rows = made_rows  # type: ignore

        # 후처리 필터가 있다면 적용
        if post_creational_filter:
            interim_rows = [
                post_creational_filter(row) for row in interim_rows
            ]
        return interim_rows
```

코드를 보면 생각보다 심플하고 내부 동작을 이해해볼수 있습니다. 일단 첫번째로 `후처리 필터(post_creational_filter)` 인데요. 이 기능은 속성앞에 `_` 가 붙은것을 보았을때
우리가 외부에서 조작하여 활용하는 속성은 아닌 것임을 알수 있고, 내부에서 나온 `row` 값을 최종적인 반환형태로 만드는데 기여합니다. 일단은 내부에서 어떠한 후처리를 통해 반환값을 사용하기 좋게 변환할 수 있는 시스템이 내재되어 있다 정도로 이해하고 넘어갑시다.


이제 다음으로 `_unique_filter_state` 인데요 이건 후처리 필터와는 살짝 다르게 외부에서 주입가능하다고 설명되어 있습니다. Python 의 `set` 중복 제거를 위한 자료구조로 이용하고 우리는 테이블의 값이 가질수 있는 고유한 값을 `strategy` 에 넘겨줌으로써 중복을 제거할 수 있다고 합니다.

```
        :param strategy: a callable that will be applied to rows or objects
         being iterated, which should return an object that represents the
         unique value of the row.   A Python ``set()`` is used to store
         these identities.   If not passed, a default uniqueness strategy
         is used which may have been assembled by the source of this
         :class:`_engine.Result` object.

        """
        self._unique_filter_state = (set(), strategy)
```

이제 `all()` 메소드를 되짚어 보면 단순하게도 모든 데이터를 가져오고, 원하는 mapping 전략이 있으면 `후필터` 를 적용해주고, unique 처리를 원한다면 `unique` 처리도 해준다음 Row 의 Sequence 를 리턴한다는 사실을 알게 되었습니다. 약간은 오버해서 깊숙하게 들어간면이 있지만, 이 모든것들이 나중에 도움이 될것이라고 생각됩니다.

In [ ]:

from sqlalchemy import select

with Session(engine) as session:
    stmt = select(User.name, User.fullname)
    users = session.execute(stmt).all()
    print(type(users)) # list
    print(user["name"]) # 공식문서에서는 named_tuple 이라고 표현하지만, functional 하게만 tuple 처럼 런타임에 동작하고 실상 타입은 Row 임

In [50]:
"""
중복처리
- 예시를 위해 동일하게 `user1` 을 가진 유저를 한명 더 추가해보도록 하겠습니다
"""
with Session(engine) as session:
    stmt = select(User).where(User.name == "user1")
    users = session.execute(stmt).all()
    print(len(users)) # 1
    print(users[0][0])

with Session(engine) as session:
    duplicated_user = User(id=101, name="user1", fullname="user1", addresses=[Address(email_address="duplicated_user@gmail.com")])
    session.add(duplicated_user)
    session.commit()

with Session(engine) as session:
    stmt = select(User).where(User.name == "user1")
    users = session.execute(stmt).all()
    print(len(users)) # 2
    print(users)

2025-05-02 00:16:00,852 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 00:16:00,854 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-05-02 00:16:00,855 INFO sqlalchemy.engine.Engine [cached since 1567s ago] ('user1',)
2
User(id=2, name='user1', fullname='User 1')
2025-05-02 00:16:00,856 INFO sqlalchemy.engine.Engine ROLLBACK
2025-05-02 00:16:00,856 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 00:16:00,857 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-05-02 00:16:00,857 INFO sqlalchemy.engine.Engine [cached since 1567s ago] ('user1',)
<class 'list'>
[(User(id=2, name='user1', fullname='User 1'),), (User(id=101, name='user1', fullname='user1'),)]
2025-05-02 00:16:00,858 INFO sqlalchemy.engine.Engine ROLLBACK


## Unique 처리

위의 예시를 보면 `user1` 이라는 name 을 가진 유저가 두명 존재하는 것을 확인할 수 있습니다. 우리는 `name` 이 동일한 두 유저를 보고 싶지 않을 경우 쿼리에서 unique 처리가 필요합니다 아까 배웠던 Unique 처리를 한번 해보도록 합시다.

In [ ]:
with Session(engine) as session:
    stmt = select(User).where(User.name == "user1")
    users = session.execute(stmt).unique(strategy=lambda row: row[0].name).all()
    print(len(users)) # 1

## Row

우리가 원하는 값에 대한 중복처리를 `unique` 와 함께 하는 법을 알아보았습니다. 다만 여기서 `lambda row: row[0]` 부분에 의문을 가지는 분들이 생기셨을 수 있다고 생각됩니다. 왜 `lambda row: row.name` 을 사용하지 못하지? 라는 의문을 충분히 가지실수 있습니다. 왜냐면 아래 코드에서는 `user.name` 이 실제로 가능하기 때문입니다. 그 이유는 무엇일까요?

```python
from sqlalchemy import select

with Session(engine) as session:
    stmt = select(User.name, User.fullname).where(User.name == "user1")
    users = session.execute(stmt).all()
    print(type(users)) # list
    for user in users:
        print(type(user)) # <class 'sqlalchemy.engine.row.Row'>
        print(user.name) # user1
```

잠깐 다른길로 새는거 같지만 이 방식을 이해하기 위해서는 `Row` 를 반드시 이해해야 합니다. 사실 우리가 하고 있는 `SELECT` 를 잘 이해하기 위해서도 필요한 과정이기에 `Row` 를 한번 짚고 넘어가도록 하겠습니다. Row 는 기본적으로 python 의 `namedtuple` 처럼 행동합니다. namedtuple 을 쉽게 이야기하면 data 구조를 조금 더 쉽게 만들기 위해, 직접 클래스(DTO) 를 만들기 보다는 tuple 의 구조를 빌리되 이름만 붙일 수 있도록 해주는 것이죠.

```python
"""
클래스를 활용한 방식
"""
class Book():
  def __init__(self, title, code):
    self.title = title
    self.code = code

book = Book(title="roach", code="1")

"""
튜플을 활용한 방식 (immutable)
"""
book = ("roach", "1")

"""
namedtuple 을 활용한 방식 (immutable)
"""
Book = namedtuple('Book', ['title', 'code'])
book = Book("roach", "1")
```

다만 기존 tuple 은 property 라는 개념이 없기 때문에 `book[0] <"roach">` 이런식으로 접근해야 했었습니다. 하지만 `namedtuple` 은 property 를 지원해주기 때문에 `book.title`, `book.code` 와 같은 방식으로 코드를 작성하는 것이 가능해졌습니다. 즉, 우리가 속성에 접근하듯 값에 접근하는 것도 가능하고, `[0]` 과 같이 인덱스 형식으로도 접근하는 것이 가능한 이유는 `Row` 가 최대한 python 의 namedtuple 처럼 행동하려 하기 때문입니다. 이러한 점을 알고 아래 내용을 함께 보도록 하겠습니다.

우리가 처음에 `Result` 코드를 분석했을때 `ResultMetadata` 라는 코드가 존재했었습니다. 이 부분이 사실 `[0]` 을 써야 하는가 안써야 하는가의 핵심적인 구분점입니다. 해당 값이 어떤 상황에 달라지는지 이해해 보도록 하겠습니다. 무턱대고 ORM 을 사용하다보면 `[0]` 을 붙여야 하는지 안붙여야 하는지 모르는 문제가 생길 수도 있습니다.

우리가 select(User.name, User.fullname) 으로 조회하게 되면 `all()` 을 호출하는 순간의 Result class 의 속성값을 살펴보면 `SimpleResultMetaData` 에 keys 의 값으로 `RMKeyView(['name', 'fullname'])` 가 포함되어 있습니다. 즉, RMKeyView 에 이미 0번이 `name`, 1번이 `fullname` 임을 알수 있죠? 그렇기에 당연하게도 `.name` 대신 `[0]` 으로, `.fullname` 대신 `[1]` 로도 접근할 수 있음을 확인할수 있어 보입니다. 한번 코드로도 확인해 보겠습니다.

```python
from sqlalchemy import select

with Session(engine) as session:
    stmt = select(User.name, User.fullname).where(User.name == "user1")
    users = session.execute(stmt).all()
    print(type(users)) # list
    for user in users:
        print(type(user)) # <class 'sqlalchemy.engine.row.Row'>
        print(user[0], user[1]) # user1 User 1
```

데이터가 아주 잘 나오는 것을 확인할 수 있습니다.

하지만 `select(User)` 를 하는 순간은 `SimpleResultMetaData` 에 keys 의 값으로 `RMKeyView(['User'])` 가 채워져 있는 것을 확인할 수 있습니다. 즉, [0] 으로 한번 접근해야만 User 객체를 얻을 수 있기 때문에 [0] 을 쓰고 name 을 써야했던 이유이죠. 그렇다면 여기서 아까 코드에서 `row.User.name` 을 하면 가능할지 살펴보도록 하겠습니다.

```python
with Session(engine) as session:
    stmt = select(User).where(User.name == "user1")
    users = session.execute(stmt).unique(strategy=lambda row: row.User.name).all()
    print(len(users)) # 1
```

잘 동작하는 것을 확인할 수 있습니다. 이제 Row 를 어떻게 써야하는지 감을 잡으셨을거라고 생각하니 다음 강의로 넘어가 보도록 하겠습니다.


In [60]:
from sqlalchemy import select

with Session(engine) as session:
    stmt = select(User.name, User.fullname).where(User.name == "user1")
    users = session.execute(stmt).all()
    print(type(users)) # list
    for user in users:
        print(type(user)) # <class 'sqlalchemy.engine.row.Row'>
        print(user.name) # user1

2025-05-02 00:42:14,120 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 00:42:14,121 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-05-02 00:42:14,122 INFO sqlalchemy.engine.Engine [cached since 2839s ago] ('user1',)
<class 'list'>
<class 'sqlalchemy.engine.row.Row'>
user1
<class 'sqlalchemy.engine.row.Row'>
user1
2025-05-02 00:42:14,124 INFO sqlalchemy.engine.Engine ROLLBACK


In [58]:
from sqlalchemy import select

with Session(engine) as session:
    stmt = select(User.name, User.fullname).where(User.name == "user1")
    users = session.execute(stmt).all()
    print(type(users)) # list
    for user in users:
        print(type(user)) # <class 'sqlalchemy.engine.row.Row'>
        print(user[0], user[1]) # user1 User 1

2025-05-02 00:27:01,892 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 00:27:01,893 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-05-02 00:27:01,894 INFO sqlalchemy.engine.Engine [cached since 1908s ago] ('user1',)
<class 'list'>
<class 'sqlalchemy.engine.row.Row'>
user1 User 1
<class 'sqlalchemy.engine.row.Row'>
user1 user1
2025-05-02 00:27:01,895 INFO sqlalchemy.engine.Engine ROLLBACK


In [59]:
with Session(engine) as session:
    stmt = select(User).where(User.name == "user1")
    users = session.execute(stmt).unique(strategy=lambda row: row.User.name).all()
    print(len(users)) # 1

2025-05-02 00:41:58,418 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-02 00:41:58,419 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-05-02 00:41:58,420 INFO sqlalchemy.engine.Engine [cached since 3154s ago] ('user1',)
1
2025-05-02 00:41:58,422 INFO sqlalchemy.engine.Engine ROLLBACK
